In [1]:
import os
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import read_csv, concat
sns.set_theme(context='notebook', style='white', font_scale=1.33)

import numpy as np
from os.path import dirname
from pandas import read_csv
import pandas as pd
from numba import njit
from tqdm import tqdm
import json

from os.path import exists
from stan_fit_scripts import fit_ppc


## Analysis

In [2]:
#set models,session to analyse
stan_models = ['m4a','m4b']
sessions = [1, 2, 3]
blocks = [1, 2, 3]



In [6]:
for stan_model in stan_models:
    for i,j in [[1,2],[2,3],[1,3]]:
        path_to_file_trt = os.path.join('stan_results',#'reliability_old',
                        f'{stan_model}_trt_{i}_{j}_summary.tsv')
        if exists(path_to_file_trt):
            print(path_to_file_trt)
            
        path_to_file_ic = os.path.join('stan_results',
                        f'{stan_model}_trt_{i}_{j}_summary.tsv')
        if exists(path_to_file_ic):
            print(path_to_file_ic)

stan_results/m4a_trt_2_3_summary.tsv
stan_results/m4a_trt_1_3_summary.tsv
stan_results/m4b_trt_1_2_summary.tsv
stan_results/m4b_trt_2_3_summary.tsv
stan_results/m4b_trt_1_3_summary.tsv


In [11]:
df = read_csv('stan_results/m4a_trt_1_3_summary.tsv', sep='\t')
df[df.R_hat>1.02].count()['R_hat']

1635

### TEST RETEST

In [4]:
# get ppc
all_data_trt = []

for stan_model in stan_models:
    for i,j in [[1,2],[2,3],[3,1]]:
        path_to_file = os.path.join('stan_results','reliability_old',
                        f'{stan_model}_trt_{i}_{j}_summary.tsv')
        if exists(path_to_file):
            df = read_csv(path_to_file, sep='\t')
            df['model'] = stan_model
            df['pair'] = str(i)+'_'+str(j)
            all_data_trt.append(df)


all_data_trt = concat(all_data_trt)



In [5]:
all_data_trt.reset_index(inplace=True)

In [6]:
all_data_trt[all_data_trt.R_hat>1.02].groupby(['model','pair']).count()['R_hat']

model  pair
m4a    1_2     201
       2_3     201
       3_1     201
m4b    1_2     161
       2_3     161
       3_1     161
Name: R_hat, dtype: int64

In [7]:
all_data_trt

,index,name,Mean,MCSE,StdDev,5%,50%,95%,N_Eff,N_Eff/s,R_hat,model,pair
0,0,lp__,-27016.800000,1.266560,28.056500,-27062.300000,-27016.400000,-26970.400000,490.70000,0.015671,1.00413,m4a,1_2
1,1,"mu_pr[1,1]",-0.513900,0.002829,0.083067,-0.651212,-0.515967,-0.370298,861.98200,0.027528,1.00066,m4a,1_2
2,2,"mu_pr[1,2]",-0.235420,0.002717,0.079499,-0.359349,-0.237810,-0.100096,856.06500,0.027339,1.00150,m4a,1_2
3,3,"mu_pr[2,1]",-0.513270,0.003438,0.094536,-0.669051,-0.512827,-0.354713,756.03100,0.024145,1.00047,m4a,1_2
4,4,"mu_pr[2,2]",-0.355550,0.003067,0.094745,-0.514841,-0.355227,-0.198632,954.52700,0.030484,1.00173,m4a,1_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10264,1512,sigma_d[4],0.007033,0.000252,0.003613,0.001554,0.006901,0.012989,206.09100,0.004178,1.01064,m4b,3_1
10265,1513,rho[1],0.008511,0.320754,0.773182,-0.959825,0.122114,0.954260,5.81056,0.000118,1.32813,m4b,3_1
10266,1514,rho[2],-0.005534,0.322274,0.764145,-0.963737,0.019430,0.951370,5.62215,0.000114,1.32217,m4b,3_1
10267,1515,rho[3],0.017478,0.283533,0.747318,-0.965457,0.140205,0.965978,6.94709,0.000141,1.29684,m4b,3_1


In [6]:


# all_data['param_name'] = all_data.name.str.split('[', expand=True)[0]
# all_data['subj_n'] = all_data.name.str.split('[', expand=True)[1]
# all_data['subj_n'] = all_data['subj_n'].str[:-1]

# all_data['ses']=np.where(all_data.subj_n.str.split(',', expand=True)[1].isna(),
#         np.nan,
#         all_data.subj_n.str.split(',', expand=True)[0])

# all_data['subj'] = np.where(all_data.subj_n.str.split(',', expand=True)[1].isna(),
#         all_data.subj_n.str.split(',', expand=True)[0],
#         all_data.subj_n.str.split(',', expand=True)[1])

# all_data['subj_ses'] = all_data['subj'] +"_"+all_data['ses'].astype('str')


In [7]:
# all_data_parmas = all_data[all_data.param_name.isin(['beta_GO','beta_P', 
#                                    'eta_gain','eta_loss'])]


In [8]:
# all_data_parmas.reset_index(inplace=True)

In [9]:
# all_data_parmas

In [10]:
# all_data_parmas.pivot(index='subj_ses', 
#                       columns=['param_name', 'Mean'],
#                       values="Mean").reset_index()


### INTERNAL CONSISTENCY

In [11]:
# get ppc
all_data_ic = []

for stan_model in stan_models:
    for i,j in [[1,2],[2,3],[3,1]]:
        path_to_file = os.path.join('stan_results',
                        f'{stan_model}_ic_{i}_{j}_summary.tsv')
        if exists(path_to_file):
            df = read_csv(path_to_file, sep='\t')
            df['model'] = stan_model
            df['pair'] = str(i)+'_'+str(j)
            all_data_sh.append(df)


all_data_sh = concat(all_data_sh)



In [13]:
all_data_sh.reset_index(inplace=True)

In [14]:
# check convergence 
all_data_sh[all_data_sh.R_hat>1.02].groupby(['model','pair']).count()['R_hat']

model  pair
m4a    1_2     78
       2_3     78
       3_1     78
m4b    1_2     71
       2_3     71
       3_1     71
Name: R_hat, dtype: int64